In [41]:
from numpy.random import seed

seed(1)
from tensorflow import set_random_seed

set_random_seed(2)
from util import read_data,read_data2, get_vocab, vectorize_sentences, indent_sentences
import numpy as np
import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.layers.core import Dense
import os
from nltk.translate.bleu_score import sentence_bleu,SmoothingFunction
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
sess = tf.Session()

In [3]:
SENTENCE_START_TOKEN = "<START>"
SENTENCE_END_TOKEN = "<EOS>"
OOV_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"

In [4]:
df = read_data()

In [5]:
# df = df[:1000]

In [6]:
msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
msk2 = np.random.rand(len(df_train)) < 0.8
df_validate = df_train[~msk2]
df_train = df_train[msk2]
df_test = df[~msk]

In [7]:
print("Train size: %s"%len(df_train))
print("Validation size: %s"%len(df_validate))
print("Test size: %s"%len(df_test))

Train size: 103155
Validation size: 25564
Test size: 32232


In [8]:
eng_vocab, rev_eng_vocab = get_vocab(df["english_sentences"], addtional_tokens=[PAD_TOKEN,OOV_TOKEN], top=None)
heb_vocab, rev_heb_vocab = get_vocab(df["hebrew_sentences"],
                                         addtional_tokens=[PAD_TOKEN,OOV_TOKEN, SENTENCE_START_TOKEN, SENTENCE_END_TOKEN],
                                         top=None)

100%|██████████| 160951/160951 [00:17<00:00, 9322.47it/s]


Original vocab size 20550


100%|██████████| 160951/160951 [00:16<00:00, 9492.77it/s] 


Original vocab size 55736


In [9]:
def vectorize_dataset(df):
    vect_eng_sentences = vectorize_sentences(df["english_sentences"], eng_vocab,encode=True,reverse=True)
    decoder_input_data = vectorize_sentences(df["hebrew_sentences"], heb_vocab, add_prefix_token=SENTENCE_START_TOKEN,encode=True)
    decoder_target_data = np.array([np.concatenate((x[1:], [heb_vocab[SENTENCE_END_TOKEN]]), axis=0) for x in decoder_input_data])
    return vect_eng_sentences,decoder_input_data,decoder_target_data

### Train data

In [10]:
train_vect_eng_sentences,train_decoder_input_data,train_decoder_target_data = vectorize_dataset(df_train)

100%|██████████| 103155/103155 [00:09<00:00, 10344.81it/s]


### Validation data

In [11]:
val_vect_eng_sentences,val_decoder_input_data,val_decoder_target_data = vectorize_dataset(df_validate)

100%|██████████| 25564/25564 [00:02<00:00, 9896.74it/s] 


### Test data

In [12]:
test_vect_eng_sentences,test_decoder_input_data,test_decoder_target_data = vectorize_dataset(df_test)

100%|██████████| 32232/32232 [00:03<00:00, 9845.80it/s] 


In [13]:
vocab_size_english = len(eng_vocab)
vocab_size_hebrew = len(heb_vocab)
max_encoder_seq_length = max([len(txt) for txt in np.append(np.append(train_vect_eng_sentences,val_vect_eng_sentences),test_vect_eng_sentences)])
max_decoder_seq_length = max([len(txt) for txt in np.append(np.append(train_decoder_input_data,val_decoder_input_data),test_decoder_input_data)])
emb_dim = 64
lstm_size = 64
batch_size = 128

In [14]:
def pad_sequences(sequence,pad_id,to_length):
    need_to_add = to_length-len(sequence)
    return np.concatenate((sequence,np.array([pad_id]*need_to_add)),axis=0)

In [15]:
train_vect_eng_sentences = np.array([pad_sequences(sentence,eng_vocab[PAD_TOKEN],max_encoder_seq_length) for sentence in train_vect_eng_sentences])
train_decoder_input_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in train_decoder_input_data])
train_decoder_target_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in train_decoder_target_data])
val_vect_eng_sentences = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_encoder_seq_length) for sentence in val_vect_eng_sentences])
val_decoder_input_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in val_decoder_input_data])
val_decoder_target_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in val_decoder_target_data])
test_vect_eng_sentences = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_encoder_seq_length) for sentence in test_vect_eng_sentences])
test_decoder_target_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in test_decoder_target_data])
test_decoder_input_data = np.array([pad_sequences(sentence,heb_vocab[PAD_TOKEN],max_decoder_seq_length) for sentence in test_decoder_input_data])


### inputs and outputs

In [16]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_inputs')
decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')

In [17]:
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')
decoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='decoder_inputs_length')

In [18]:
learning_rate = tf.placeholder(tf.float32, shape=[])

### embeddings

In [19]:
embedding_encoder = tf.get_variable("embedding_encoder", [vocab_size_english, emb_dim])

In [20]:
encoder_emb_inp = tf.nn.embedding_lookup(embedding_encoder, encoder_inputs)

In [21]:
embedding_decoder = tf.get_variable("embedding_decoder", [vocab_size_hebrew, emb_dim])

In [22]:
decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder, decoder_inputs)

In [23]:
lstm_initializer = tf.random_uniform_initializer(-0.08,0.08)

### encoder

In [24]:
encoder_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(lstm_size,initializer=lstm_initializer) for _ in range(2)])

In [25]:
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
    encoder_cell, encoder_emb_inp,sequence_length=encoder_inputs_length,dtype=tf.float32, time_major=True)

### decoder

In [26]:
decoder_cell = tf.contrib.rnn.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(lstm_size,initializer=lstm_initializer) for _ in range(2)])

In [27]:
decoder_outputs, decoder_final_state = tf.nn.dynamic_rnn(
    decoder_cell, decoder_emb_inp,initial_state=encoder_state,sequence_length=decoder_inputs_length,
    dtype=tf.float32, time_major=True, scope="plain_decoder",
)

In [28]:
decoder_logits = tf.contrib.layers.linear(decoder_outputs, vocab_size_hebrew)

In [29]:
decoder_prediction = tf.argmax(decoder_logits, 2)

### loss and train

In [30]:
stepwise_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(decoder_targets, depth=vocab_size_hebrew, dtype=tf.float32),
                                                                 logits=decoder_logits)
loss = tf.reduce_mean(stepwise_cross_entropy)
tf.summary.scalar('loss', loss)
optimizer = tf.train.AdamOptimizer()
gvs = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
train_op = optimizer.apply_gradients(capped_gvs)

In [31]:
merged = tf.summary.merge_all()

In [32]:
train_writer = tf.summary.FileWriter('tensorboard/train',sess.graph)

### train

In [33]:
sess.run(tf.global_variables_initializer())

In [34]:
saver = tf.train.Saver()

In [35]:
if not os.path.exists("checkpoints"):
    os.mkdir("checkpoints")
latest_model = tf.train.latest_checkpoint("checkpoints")

In [36]:
if latest_model is not None:
    saver.restore(sess, "model.ckpt")

In [37]:
def get_batch(enc_input,dec_input,dec_target,batch_size):
    number_of_batches = int(len(enc_input)/batch_size)
    counter = 0
    while True:
        from_index = (counter%number_of_batches)*batch_size
        to_index = ((counter%number_of_batches)+1)*(batch_size)
        feed = {}
        if enc_input is not None:
            feed[encoder_inputs]=enc_input[from_index:to_index].T
            feed[encoder_inputs_length] = np.array([len([word for word in sequence if word!=0]) for sequence in enc_input[from_index:to_index]],dtype=np.int32)
        if dec_input is not None:
            feed[decoder_inputs]=dec_input[from_index:to_index].T
            feed[decoder_inputs_length] = np.array([len([word for word in sequence if word!=0]) for sequence in dec_input[from_index:to_index]],dtype=np.int32)
        if dec_target is not None:
            feed[decoder_targets] = dec_target[from_index:to_index].T
        yield feed
        counter += 1

In [38]:
def decode_sequence(sequence,rev_vocab,to_join=True):
    arr = [rev_vocab[int(index)] for index in sequence if rev_vocab[int(index)]!="<PAD>" and rev_vocab[int(index)]!="<EOS>"]
    if to_join: 
        return " ".join(arr)
    else:
        return arr

In [39]:
def get_bleu_summary(value,iteration):
    bleu_summary = tf.Summary(value=[tf.Summary.Value(tag="validation_bleu", simple_value=value), ])
    return bleu_summary

In [49]:
chencherry = SmoothingFunction()
def get_validation_bleu_scores(sess,generator,iterations):
    scores = []
    for i in range(iterations):
        feed = next(generator)
        validation_predict_ = sess.run(decoder_prediction, feed)
        scores += [sentence_bleu([decode_sequence(pred,rev_heb_vocab,False)],decode_sequence(exp,rev_heb_vocab,False),smoothing_function=chencherry.method1) for pred,exp in zip(validation_predict_.T,feed[decoder_targets].T)]
    return np.mean(scores)

In [43]:
loss_track = []

In [44]:
max_batches = 20000
batches_per_epoch = int(len(train_vect_eng_sentences)/batch_size)
batches_per_epoch_validation = int(len(val_vect_eng_sentences)/batch_size)
log_every_iterations = 50
print_samples_every_iterations = 250
train_feed_generator = get_batch(train_vect_eng_sentences,train_decoder_input_data,train_decoder_target_data,batch_size)
validation_feed_generator = get_batch(val_vect_eng_sentences,val_decoder_input_data,val_decoder_target_data,batch_size)
try:
    for batch in range(max_batches):
        if batch%batches_per_epoch==0:
            print("Epoch %s"%(int(batch/batches_per_epoch)))
        fd = next(train_feed_generator)
        _, l,summary = sess.run([train_op, loss,merged], fd)
        train_writer.add_summary(summary, batch)
        loss_track.append(l)
        if batch == 0 or batch % log_every_iterations == 0:
            print('  minibatch loss: {}'.format(l))
            mean_validation_bleu = get_validation_bleu_scores(sess,validation_feed_generator,batches_per_epoch_validation)
            train_writer.add_summary(get_bleu_summary(mean_validation_bleu,batch), batch)
            saver.save(sess,"checkpoints/model",global_step=batch)
            if batch % print_samples_every_iterations == 0:
                predict_ = sess.run(decoder_prediction, fd)
                for i, (inp, pred,exp) in enumerate(zip(fd[encoder_inputs].T, predict_.T,fd[decoder_targets].T)):
                    print('  sample {}:'.format(i + 1))
                    print('    input     > {}'.format(decode_sequence(inp[::-1],rev_eng_vocab)))
                    print('    predicted_raw > {}'.format(pred))
                    print('    expected_raw > {}'.format(exp))
                    print('    predicted > {}'.format(decode_sequence(pred,rev_heb_vocab)))
                    print('    expected > {}'.format(decode_sequence(exp,rev_heb_vocab)))
                    if i >= 2:
                        break
                print()
except KeyboardInterrupt:
    print('training interrupted')

Epoch 0
  minibatch loss: 10.928521156311035
  sample 1:
    input     > i have to go to sleep .
    predicted_raw > [3 3 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    expected_raw > [   6.   38.   85.  317.    4.    3.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0. 

   0.00000000e+00]
    predicted > 
    expected > אין לי רעיה .

  minibatch loss: 10.118772506713867
  minibatch loss: 10.028480529785156
  minibatch loss: 9.920879364013672
  minibatch loss: 9.83253288269043
  minibatch loss: 9.727063179016113
  sample 1:
    input     > tom could not convince mary .
    predicted_raw > [4 4 4 4 3 4 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    expected_raw > [   9.    8.  463.  487.    5.   45.    4.    3.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.

    expected > תום לא אהב את זה במיוחד .
  sample 2:
    input     > tom did not like it much .
    predicted_raw > [6 8 8 4 4 4 4 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    expected_raw > [    9.     8.  1110.     5.    10.    62.     4.     3.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.

     0.     0.     0.     0.     0.]
    predicted > הוא הוא את את
    expected > במקומך הייתי ממתין .

  minibatch loss: 8.708783149719238
  minibatch loss: 8.62993049621582
  minibatch loss: 8.519994735717773
  minibatch loss: 8.430745124816895
  minibatch loss: 8.334268569946289
  sample 1:
    input     > he lay down on his back .
    predicted_raw > [ 6  8 11 11  4  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
    expected_raw > [   12.  2851.    13.  2100.     4.     3.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.  

   0.00000000e+00]
    predicted > תום לא לא את . .
    expected > הכרטיסים נמכרים מהר והמקום מוגבל .
  sample 3:
    input     > delicious refreshments were served .
    predicted_raw > [9 8 4 4 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    expected_raw > [  2.55200000e+03   1.00620000e+04   2.06560000e+04   4.00000000e+00
   3.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.

...

   0.00000000e+00]
    predicted > תום אמר שהוא אכל כל יום .
    expected > תום אמר שהוא בכה כל יום .

  minibatch loss: 0.1225166916847229
Epoch 24
  minibatch loss: 0.2195582091808319
  minibatch loss: 0.15713970363140106
  minibatch loss: 0.17092429101467133
  minibatch loss: 0.1452307552099228
  sample 1:
    input     > she gave an amateur performance on the piano .
    predicted_raw > [  16 1173   44  547   13   45    4    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    

   0.00000000e+00]
    predicted > יפן שלה מוכר מטורפים .
    expected > הרעיונות שלה נשמעים משוגעים .

  minibatch loss: 0.17045587301254272
  minibatch loss: 0.15697705745697021
  minibatch loss: 0.14529506862163544
  minibatch loss: 0.10069962590932846
  minibatch loss: 0.14356698095798492
  sample 1:
    input     > i love cheese .
    predicted_raw > [   6   54 1934    4    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
    exp

   0.00000000e+00]
    predicted > לעולם לא ביקשתי לך , תום .
    expected > לעולם לא אסלח לך , תום .
  sample 2:
    input     > relax . he is just teasing you .
    predicted_raw > [2035   11   12   53 8435   60    4    3    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
    expected_raw > [  1.18490000e+04   1.10000000e+01   1.20000000e+01   5.42000000e+02
   8.43500000e+03   6.00000000e+01   4.00000000e+00   3.00000000e+00
   0.00000000e+00   0.000

   0.00000000e+00]
    predicted > אני תמיד את מעוף מהמלחינים של שונות ביותר שלו .
    expected > אני מעריך אותו כאחד מהמלחינים המודרניים הטובים ביותר ביפן .
  sample 2:
    input     > i made him go .
    predicted_raw > [1212   56   85    4    3    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
    expected_raw > [  2.40230000e+04   5.60000000e+01   8.50000000e+01   4.00000000e+00
   3.00000000e+00   0.00000000e+00   0.00000000e+00   0

     0     0     0     0     0]
    expected_raw > [  1.50000000e+01   1.28000000e+02   2.20000000e+01   5.13700000e+03
   4.39060000e+04   9.00000000e+01   3.19000000e+02   7.00000000e+00
   3.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00 

In [46]:
test_feed_generator = get_batch(test_vect_eng_sentences,test_decoder_input_data,test_decoder_target_data,batch_size)

In [51]:
number_of_batches_in_test = int(len(test_vect_eng_sentences)/batch_size)

In [52]:
for i in range(number_of_batches_in_test):
    fd = next(test_feed_generator)
    predict_ = sess.run(decoder_prediction, fd)
    test_sentences = []
    for i, (inp, pred,exp) in enumerate(zip(fd[encoder_inputs].T, predict_.T,fd[decoder_targets].T)):
        input_sentence = decode_sequence(inp[::-1],rev_eng_vocab)
        output_sentence = decode_sequence(pred,rev_heb_vocab)
        expected_sentence = decode_sequence(exp,rev_heb_vocab)
        score = sentence_bleu([decode_sequence(pred,rev_heb_vocab,False)],decode_sequence(exp,rev_heb_vocab,False),smoothing_function=chencherry.method1)
        test_sentences.append([input_sentence,output_sentence,expected_sentence,score])

In [54]:
import pandas as pd
test_df = pd.DataFrame(test_sentences,columns=["english_sentence","predicted_translation","expected_translation","bleu_score"])

In [56]:
test_df.to_csv("test_results.csv",encoding="utf-8",index=False)